# Mapping to the Qubit Space

The problems and operators with which you interact in Qiskit Nature (usually) need to be mapped into the qubit space before they can be solved with our quantum algorithms. This task is handled by the various `QubitMapper` classes.

In this tutorial, you will learn about the various options available to you.

## Fermionic Mappers

This section deals with fermionic mappers, which transform fermionic operators into the qubit space.
This is mostly used by the [electronic structure stack](01_electronic_structure.ipynb) but also finds application for the [`FermiHubbardModel`](TODO).

There exist different mapping types with different properties. Qiskit Nature already supports the following fermionic mappings:

* Jordan-Wigner (Zeitschrift für Physik, 47, 631-651 (1928))
* Parity (The Journal of chemical physics, 137(22), 224109 (2012))
* Bravyi-Kitaev (Annals of Physics, 298(1), 210-226 (2002))

We will discuss some of these in the following sections. You should learn all the information necessary, to comfortable work with any of the available mappers.

In order to discuss the various mappings, we will be using the electronic structure Hamiltonian of the H2 molecule. For more information on how to obtain this, please refer to the [electronic structure tutorial](01_electronic_structure.ipynb).

In [1]:
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver()
problem = driver.run()
fermionic_op = problem.hamiltonian.second_q_op()

### The Jordan-Wigner Mapping

The Jordan-Wigner mapping is the most straight-forward mapping with the simplest physical interpretation, because it maps the occupation of one spin-orbital to the occupation of one qubit.

<img src="aux_files/jw_mapping.png" width="500">

You can construct use it like so:

In [2]:
from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

In [3]:
qubit_jw_op = mapper.map(fermionic_op)
print(qubit_jw_op)

-0.8105479805373283 * IIII
+ 0.17218393261915543 * IIIZ
- 0.2257534922240237 * IIZI
+ 0.12091263261776633 * IIZZ
+ 0.17218393261915543 * IZII
+ 0.16892753870087907 * IZIZ
+ 0.045232799946057826 * YYYY
+ 0.045232799946057826 * XXYY
+ 0.045232799946057826 * YYXX
+ 0.045232799946057826 * XXXX
- 0.22575349222402363 * ZIII
+ 0.1661454325638241 * ZIIZ
+ 0.16614543256382408 * IZZI
+ 0.1746434306830045 * ZIZI
+ 0.12091263261776633 * ZZII


True


### The Parity Mapping

The Parity mapping is the dual mapping to the Jordan-Wigner one, in the sense that it encodes the parity information locally on one qubit, whereas the occupation information is delocalized over all qubits.

In [4]:
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper()

In [5]:
qubit_p_op = mapper.map(fermionic_op)
print(qubit_p_op)

-0.8105479805373283 * IIII
+ 0.17218393261915543 * IIIZ
- 0.2257534922240237 * IIZZ
+ 0.12091263261776633 * IIZI
+ 0.17218393261915543 * IZZI
+ 0.16892753870087907 * IZZZ
+ 0.045232799946057826 * ZXIX
- 0.045232799946057826 * IXZX
- 0.045232799946057826 * ZXZX
+ 0.045232799946057826 * IXIX
- 0.22575349222402363 * ZZII
+ 0.1661454325638241 * ZZIZ
+ 0.16614543256382408 * IZIZ
+ 0.1746434306830045 * ZZZZ
+ 0.12091263261776633 * ZIZI


This has one major benefit for the case of problems in which we want to preserve the number of particles of each spin species; it allows us to remove 2 qubits, because the information in them becomes redundant.
Since Qiskit Nature arranges the qubits in block-order, such that the first half encodes the alpha-spin, and the second half the beta-spin information, this means we can remove the N/2-th and N-th qubit.

To do this, you need to specify the number of particles in your system, like so:

In [6]:
mapper = ParityMapper(num_particles=problem.num_particles)

In [7]:
qubit_op = mapper.map(fermionic_op)
print(qubit_op)

-1.0523732457728605 * II
+ 0.39793742484317896 * IZ
- 0.39793742484317896 * ZI
- 0.01128010425623538 * ZZ
+ 0.18093119978423122 * XX


### More advanced qubit reductions

It is also possible to perform more advanced qubit reductions, which are based on finding Z2 symmetries in the Hilbert space of the qubit. A requirement for this to be useful, is that you know in which symmetry-subspace you need to look for your actual solution of interest. This can be a bit tricky, but luckily the problem classes of Qiskit Nature provide you with a utility to automatically determine that correct subspace.

Here is how you can use this to your advantage:

In [8]:
tapered_mapper = problem.get_tapered_mapper(mapper)
print(type(tapered_mapper))

<class 'qiskit_nature.second_q.mappers.tapered_qubit_mapper.TaperedQubitMapper'>


In [9]:
qubit_op = tapered_mapper.map(fermionic_op)
print(qubit_op)

-1.041093141516625 * I
- 0.7958748496863577 * Z
- 0.1809311997842312 * X


As you can see here, the H2 molecule is such a simple system that we can simulate it entirely on a single qubit!

### Interleaved ordering

As mentioned previously, Qiskit Nature arranges the fermionic spin-up and spin-down parts of the qubit register in block-order. However, sometimes one may want to interleave the registers instead. This can be achieved by means of the `InterleavedQubitMapper`. This can be shown best upon inspection of the `HarteeFock` initial state circuit:

In [10]:
from qiskit_nature.second_q.circuit.library import HartreeFock

In [11]:
hf_state = HartreeFock(2, (1, 1), JordanWignerMapper())
hf_state.draw()

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────

In [12]:
from qiskit_nature.second_q.mappers import InterleavedQubitMapper

In [13]:
interleaved_mapper = InterleavedQubitMapper(JordanWignerMapper())

In [14]:
hf_state = HartreeFock(2, (1, 1), interleaved_mapper)
hf_state.draw()

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────
          
q_3: ─────

In [15]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright